<font color="#D31525"><h3 align="left">Detección de fraude en transacciones financieras</h3></font>
<font color="#2C3E50"><h3 align="left">GENERACIÓN DE NUEVAS VARIABLES</h3></font>

## Importar librerias
En esta primera parte del código, se realizan las llamadas a las librerías que se utilizarán en el Notebook:

In [1]:
# Paquetes de manipulación de datos
import pandas as pd
import numpy as np
import boto3

# Paquetes de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Paquete de manipulación de fechas
import datetime as dt


## Importar DataSet
Una vez que hemos limpiado el dataset, realizamos el estudio descriptivo y discovery de los datos. Para ello, importamos el dataset limpio:

In [2]:
s3 = boto3.client("s3")

In [3]:
# Seleccionamos el bucket con el que vamos a trabajar
BUCKET_NAME = 'tfmfraud'

In [4]:
# Descargamos el fichero del bucket de s3 a la máquina EC2 para poder trabajar con él.
s3.download_file(Bucket = BUCKET_NAME, Key = 'df_clean.csv',Filename = '/tmp/df_clean.csv')

In [5]:
#Leemos el fichero y lo metemos en un dataframe.
df = pd.read_csv('/tmp/df_clean.csv', dtype={'rank':'category'})

In [6]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [7]:
df.shape

(6362620, 11)

In [8]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/df_clean.csv

## Generación de nuevas variables

Una vez realizado el análisis de __calidad y discovery del dato__ procedemos a crear nuevas variables que consideramos pueden ser interesantes en nuestro estudio:
1. Generamos variables de tiempo a nivel de día y semana a partir de la variable *step*
2. Generamos la variable amount_illegal para identificar aquellas transacciones que superan 200.000 u.m.
3. Diferenciamos la tipología del emisor receptor (M = 'Merchant', C = 'Client')

**Variables de tiempo**  
La variable *step* indica el momento de tiempo en que se realiza la transacción. Esta variable mapea 1 hora de tiempo real. El dataset contiene información de 31 días, por lo que en el data set encontramos 743 steps.


Calculo de la variable a nivel de día:

In [9]:
df["day"] = ((df["step"]-1)/24 + 1).astype(int)

Calculo de la variable a nivel de semana:

In [10]:
df["week"] = ((df["day"])/8 + 1).astype(int)

In [11]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1


**Variable que identifica las transacciones ilegales**  
Se considera una transacción ilegal aquella que supera la cantidad de 200.000 um. En este caso, generaremos una variable tipo indicador que identifique cuándo se produce una transacción ilegal (1) o no (0)


In [12]:
df['isFlaggedFraud_New'] = np.where(df['amount']<= 199999.99, 0, 1)

In [14]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1,0


In [13]:
df.loc[df["isFlaggedFraud_New"] == 1].head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New
15,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.00,51513.44,0.0,0.0,1,1,1
19,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.00,0.00,0.0,0.0,1,1,1
24,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.00,2719172.89,0.0,0.0,1,1,1
82,1,TRANSFER,224606.64,C873175411,0.0,0.0,C766572210,354678.92,0.00,0.0,0.0,1,1,1
84,1,TRANSFER,379856.23,C1449772539,0.0,0.0,C1590550415,900180.00,19169204.93,0.0,0.0,1,1,1


**Variable que identifica el tipo de receptor**  
En el df encontramos diferentes tipos de usuarios receptores de las transacciones (usuarios emisores solo hay de tipo 'C'):
* Usuarios tipo C - usuarios particulares
* Usuarios tipo M - usuarios comerciantes

En este caso vamos a crear una variable tipo indicador que identifique los casos en los que el receptor sea 'M' (1) o 'C' (0)


Tipología de usuarios receptores

In [15]:
df["nameDest"].groupby(df["nameDest"].str.slice(stop=1)).count()

df['ind_merchant'] = np.where(df["nameDest"].str.slice(stop=1) == 'C', 0, 1)

In [16]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1,0,1
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1,0,1
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1,0,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1,0,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1,0,1


**Variable que identifica la diferencia de los balances tanto del origen como del receptor**  
Generar una variable que sean los balances de tanto las cuentas de Origen como de Destino. Esto nos va a servir, para poder chequear los siguientes casos:
1. Casos en los que amount esté bien calculado. Para esto se tiene que cumplir que el valor de amount, balance cuenta Origen y balance cuenta destino sea el mismo.
2. Casos en los que el valor de la variable amount no cuadre con alguno de los campos nuevos generados: Balance cuenta origen y balance cuenta destino.
3. Casos en los que el balance de cuenta origen y destino no cuadre. **Hay que tener en cuenta que las transacciones cuyo destinatario sea un comercio el valor de su campo balance va a ser 0**


In [17]:
df['balanceOrig'] = df['oldbalanceOrg'] - df['newbalanceOrig']
df['balanceDest'] = df['newbalanceDest'] - df['oldbalanceDest']

In [18]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1,0,1,9839.64,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1,0,1,1864.28,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1,0,0,181.00,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1,0,0,181.00,-21182.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1,0,1,11668.14,0.0


**Variable que identifica la hora del día en la que se está.**  
Esto nos va a permitir conocer como se distribuyen las transacciones durante las horas del día.
    

In [19]:
df["hours_day"] = (df["step"] % 24)
df["hours_day"].replace({0: 24}, inplace=True)

In [20]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest,hours_day
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1,0,1,9839.64,0.0,1
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1,0,1,1864.28,0.0,1
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1,0,0,181.00,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1,0,0,181.00,-21182.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1,0,1,11668.14,0.0,1


**Variable que identifica la hora del día en la que se está.**  
Esto nos va a permitir conocer como se distribuyen las transacciones durante las horas del día.
    

In [21]:
# Calculamos los tramos de subdivisión:
bins = (0, 200000, 400000, 9.244552e+07)

In [22]:
# Categorizamos con la función cut
df['amount_category'] = pd.cut(df['amount'], bins, include_lowest=True)

In [23]:
pd.value_counts(df['amount_category'])

(-0.001, 200000.0]        4689050
(200000.0, 400000.0]      1144175
(400000.0, 92445520.0]     529395
Name: amount_category, dtype: int64

**4. Guardamos el nuevo dataset**  
Guardamos el fichero usando en nuestro bucket de s3 usando la libreria **boto3**

In [24]:
df.to_csv('/tmp/df_new_var.csv', index = False)

In [25]:
s3.upload_file(Bucket = BUCKET_NAME, Key = 'df_new_var.csv', Filename = '/tmp/df_new_var.csv')

In [25]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/df_new_var.csv